## Weakly Connected Components und Strongly Connected Components von Neo4j

WCC: Identifiziert Gruppen von Knoten in einem ungerichteten Graphen, die durch Pfade miteinander verbunden sind, unabhängig von der Richtung der Beziehungen. <br>
SCC: Identifiziert Gruppen von Knoten in einem gerichteten Graphen, in dem für jedes Knotenpaar in der Gruppe ein Pfad von jedem Knoten zum anderen existiert, wobei die Richtung der Beziehungen berücksichtigt wird. <br>

In [1]:
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "")) 

# Cypher-Abfrage für den WCC Algorithmus
wcc_query = """
CALL gds.wcc.stream('graphAcc20')
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).accountNumber AS account, componentId
ORDER BY componentId
"""

results = graph.run(wcc_query).data()

for result in results:
    print(result)

{'account': 'A1', 'componentId': 0}
{'account': 'A2', 'componentId': 0}
{'account': 'A3', 'componentId': 0}
{'account': 'A4', 'componentId': 0}
{'account': 'A5', 'componentId': 0}


In [2]:
import openai

class OpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key

    def generate_chat_response(self, prompt, model="gpt-3.5-turbo", temperature=0.7, max_tokens=150):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message['content'].strip() 
        except Exception as e:
            return f"An error occurred: {str(e)}"

In [3]:
openai_wrapper = OpenAIWrapper(api_key='')

prompt_intro = "Based on the following weakly connected components analysis of a network, where every person is connected to every other person through some path:\n\n"

prompt_body = "\n".join([f"{res['account']} is in component {res['componentId']}" for res in results])

prompt_questions = "\n\nWhat insights can we derive from this complete connectivity? What might be the implications for information flow or social dynamics within this network?"

complete_prompt = prompt_intro + prompt_body + prompt_questions

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")

Based on the following weakly connected components analysis of a network, where every person is connected to every other person through some path:

A1 is in component 0
A2 is in component 0
A3 is in component 0
A4 is in component 0
A5 is in component 0

What insights can we derive from this complete connectivity? What might be the implications for information flow or social dynamics within this network?
GPT-3's Insights: An error occurred: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.


In [6]:
scc_query = """
CALL gds.alpha.scc.stream('graphAcc20')
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).accountNumber AS account, componentId
ORDER BY componentId
"""

results = graph.run(scc_query).data()

for result in results:
    print(result)


{'account': 'A1', 'componentId': 0}
{'account': 'A2', 'componentId': 0}
{'account': 'A3', 'componentId': 0}
{'account': 'A4', 'componentId': 0}
{'account': 'A5', 'componentId': 0}


Output Deutung: <br>
alle Knoten dieselbe componentId (0) für beide Algorithmen haben, deutet dies darauf hin, dass Ihr Graph im Kontext von WCC als vollständig verbunden betrachtet wird. <br>
Für SCC bedeutet es, dass, trotz der Berücksichtigung der Richtung der Beziehungen, immer noch jeder Knoten von jedem anderen Knoten aus erreichbar ist. Dies zeigt eine starke interne Vernetzung <br>

In [7]:
prompt_intro = "Based on the analysis of strongly connected components in the network, where each person can reach every other person through a directed path, the following connections were identified:\n\n"

# Die Ergebnisse in einen lesbaren Text umwandeln
prompt_body = "\n".join([f"{res['account']} is in strongly connected component {res['componentId']}" for res in results])

prompt_questions = "\n\nConsidering this strong interconnectivity, what can be inferred about the network's resilience and potential points of failure? Additionally, how might this connectivity influence the spread of information or behaviors within the network?"

complete_prompt = prompt_intro + prompt_body + prompt_questions

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")


Based on the analysis of strongly connected components in the network, where each person can reach every other person through a directed path, the following connections were identified:

A1 is in strongly connected component 0
A2 is in strongly connected component 0
A3 is in strongly connected component 0
A4 is in strongly connected component 0
A5 is in strongly connected component 0

Considering this strong interconnectivity, what can be inferred about the network's resilience and potential points of failure? Additionally, how might this connectivity influence the spread of information or behaviors within the network?
GPT-3's Insights: An error occurred: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.o